# Programming Project - Unit 2
*by Débora Azevedo, Eliseu Jayro, Francisco de Paiva and Igor Brandão*

**Goals**
The purpose of this project is explore the following:

- Access Health Graph API - Runkeeper content;
- Geolocation analysis and hypotheses should be explained in detail;

<hr>

# Global Imports section

Import the necessary libraries to handle 

- Geocoding;
- Maps;
- File input;
- Heatmap;
- Numpy library;
- Tqdm progress bar

In [ ]:
### Library necessary to run this IPython Notebook
!pip install geocoder
!pip install folium
!pip install tqdm
!pip install tabulate
!pip install pandas-datareader
!pip install requests

In [ ]:
# Import pandas
import pandas as pd

# Import json
import json

# Import google geocoder
import geocoder as gc

# Import numpy library
import numpy as np

# Import folium maps
import os
import folium
from folium import plugins

# Import tqdm progressing bar plugin
from tqdm import tqdm

<hr>

# I - Maps section

## Geolocation data import [optional]

In [ ]:
# Import the geolocation.csv data: data
geolocation_data = pd.read_csv("geolocation.csv", encoding = 'latin2')

### Coordinate groupping

The cell below perform the latitude/longitude reading and count all og them to know if there are repetitions and group *[latitude, longitude, count]* data into the coordinate array

In [ ]:
# =================================================================================
# Data adjusts
# =================================================================================

# Count the same latitude/longitude occurrences
geodata = geolocation_data.groupby(['longitude', 'latitude']).size().reset_index().rename(columns={0:'count'})

# =================================================================================
# Add the coordinates
# =================================================================================

# Initialize the coordinates array
coordinates = []

# Add the coordinates to the coordinate
for i in tqdm(range(len(geodata))):
    # eliminate items with'nan' element
    if all(~np.isnan([geodata.loc[geodata.index[i],'latitude'], geodata.loc[geodata.index[i],'longitude']])):
        coordinates.append([geodata.loc[geodata.index[i],'latitude'], geodata.loc[geodata.index[i],'longitude'], 
                            geodata.loc[geodata.index[i],'count']])

# Display the imported coordinates
coordinates

## Add the coordinates to the maps [required]

In [ ]:
# =================================================================================
# Map settings
# =================================================================================

# Set map center and zoom level
mapc = [-5.788, -35.202]
zoom = 11

# Wind rose
url = ('https://raw.githubusercontent.com/SECOORA/static_assets/'
       'master/maps/img/rose.png')

# Create HeatMap instance
htMap = folium.Map(location=mapc, zoom_start=zoom, tiles='OpenStreetMap')

# Create marker map instance
mkMap = folium.Map(location=mapc, zoom_start=zoom, tiles='OpenStreetMap')

# Create circle marker map instance
circMap = folium.Map(location=mapc, zoom_start=zoom, tiles='OpenStreetMap')

# Chunk size
chunksize = 10 ** 8 # can be changed

# For item i in a range that is a length of l,
for i in range(0, len(coordinates), chunksize):
    # Append the coordinates to the Heat Map
    plugins.HeatMap(coordinates[i:i+chunksize]).add_to(htMap)
    plugins.FloatImage(url, bottom=5, left=75).add_to(htMap)
    
    # Append the coordinates to Marker Map
    plugins.MarkerCluster(coordinates[i:i+chunksize]).add_to(mkMap)
    plugins.FloatImage(url, bottom=5, left=75).add_to(htMap)

## 1) Heat Map

In [ ]:
# Print the heatMap
htMap

## 2) Cluster Map

In [ ]:
# Print the cluster map
mkMap

## 3) With Natal Neighborhoods

In [ ]:
# import geojson file about natal neighborhoods
natal_neigh = os.path.join('geojson', 'natal.geojson')

# load the data and use 'UTF-8'encoding
geo_json_natal = json.load(open(natal_neigh,encoding='UTF-8'))

In [ ]:
# print the keys of the dictionary
print(geo_json_natal.keys())
# print the list of features (neighborhoods)
geo_json_natal['features']

In [ ]:
neighborhood = []
# list all neighborhoods
for neigh in geo_json_natal['features']:
        neighborhood.append(neigh['properties']['name'])

### 3.1) Heatmap with Natal neighborhoods

In [45]:
# Create new map instance
htMapNeighborhood = htMap

# Configure geojson layer
folium.GeoJson(geo_json_natal).add_to(htMapNeighborhood)

# print the map
htMapNeighborhood

## 4) Choropleth map

### Service to convert lat/lng to neighborhood [It consumes external API - too heavy!]

Just execute this cell if the file *neighborFrequency.csv* wasn't generated before

In [ ]:
# =================================================================================
# Coordinate reversing
#
# convert lat/lng to neighborhood
# =================================================================================
import requests
import json
from pandas.io.json import json_normalize

# Access token
API_KEY = '574ad2a81d281f'

# Function to get the neighborhood from lat/lng
def getplace(lat, lng):
    # Base URI
    url = "https://us1.locationiq.com/v1/reverse.php"

    # Parameter data
    data = {
        'key': API_KEY,
        'lat': lat,
        'lon': lng,
        'format': 'json'
    }

    # Perform the request
    response = requests.get(url, params=data).json()
    
    # Check the return
    if 'address' in response:
        return response['address']['suburb']
    else:
        return ''

# =================================================================================

# Copy the data
neighborFrequency = pd.DataFrame()

# Set a limit
# LIMIT = len(coordinates)
LIMIT = 100

# Perform a massive convertion from lat/lng to suburb
for i in tqdm(range(LIMIT)):
    # Call the conversion function
    itemData = {
        'district': getplace(coordinates[i][0], coordinates[i][1]),
        'count': 1,
    }

    # Convert the data to json
    itemData = json_normalize(itemData)

    # Add the data to neighborhood dataFrame
    neighborFrequency = pd.concat([neighborFrequency, itemData])

# Group the neighbor data
neighborFrequency = neighborFrequency.groupby(['district']).size().reset_index().rename(columns={0:'count'})

# Export the neighbor data to csv
neighborFrequency.to_csv('neighborFrequency.csv', encoding="utf-8")

### Choropleth map color scale

In [ ]:
from branca.colormap import linear

colormap = linear.YlGn_09.scale(
    neighborFrequency['count'].min(),
    neighborFrequency['count'].max())

print(colormap(1.0))

colormap

In [38]:
neighbor_dict = neighborFrequency.set_index('district')['count']

# Function to get the neighborhood from lat/lng
def applyColor(district):
    try:
        return colormap(neighbor_dict[district])
    except KeyError:
        return '#ffffe5'

In [44]:
# =================================================================================
# Map settings
# =================================================================================

# Set map center and zoom level
mapc = [-5.788, -35.202]
zoom = 11

# definition of the boundaries in the map
geo_json_data = json.load(open(natal_neigh,encoding='UTF-8'))

# creation of the choropleth
choroplethNatal = folium.Map(location=mapc, zoom_start=zoom, tiles='OpenStreetMap')

# Wind rose
url = ('https://raw.githubusercontent.com/SECOORA/static_assets/'
       'master/maps/img/rose.png')

# set the map data
folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': applyColor(feature['properties']['name']),
        'key_on': 'feature.properties.name',
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.7,
        'legend_name': 'Exercises in Natal'
    }
).add_to(choroplethNatal)

plugins.FloatImage(url, bottom=5, left=75).add_to(choroplethNatal)
               
# print the map
choroplethNatal